Installation and Repository cloning

In [1]:
!pip install adversarial-robustness-toolbox --quiet
!pip install multiprocess --quiet
!pip install importlib --quiet
!git clone https://github.com/Georgsiedel/corruption-testing.git --quiet

Import Libraries

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import numba
numba.__version__

import importlib
import time
import matplotlib.pyplot as plt
import numpy as np

from art.estimators.classification import PyTorchClassifier
from PIL import Image, ImageDraw, ImageFont, ImageOps

Import Adversarial Attack Methods

Load and Prepare CIFAR-10 Dataset

In [3]:
def load_dataset(dataset_split: int):
  # Load CIFAR-10 dataset using torchvision
  transform = transforms.Compose([transforms.ToTensor()])
  testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

  # Truncated testset for experiments and ablations
  truncated_testset, _ = torch.utils.data.random_split(testset,
                                                      [dataset_split, len(testset) - dataset_split],
                                                      generator=torch.Generator().manual_seed(42))

  # Extract data and labels from torchvision dataset
  xtest = torch.stack([data[0] for data in truncated_testset])
  ytest = torch.tensor([data[1] for data in truncated_testset])

  return xtest, ytest

Load and Prepare WideResNet Model

In [4]:
import models.wideresnet as wideresnet

modeltype = 'robust'

if modeltype == 'standard':
    net = wideresnet.WideResNet_28_4(10, 'CIFAR10', normalized=True, block=wideresnet.WideBasic, activation_function='relu')
elif modeltype == 'robust':
    net = wideresnet.WideResNet_28_4(10, 'CIFAR10', normalized=True, block=wideresnet.WideBasic, activation_function='silu')

net = torch.nn.DataParallel(net)
PATH = f'./models/pretrained_models/{modeltype}.pth'
state_dict = torch.load(PATH)
net.load_state_dict(state_dict["model_state_dict"], strict=False)
net.eval()

DataParallel(
  (module): WideResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (layer1): Sequential(
      (0): WideBasic(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (dropout): Dropout(p=0.0, inplace=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (shortcut): Sequential(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (1): WideBasic(
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (dropout): Dropout(p=0.0, inplace=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [5]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

# Initialize PyTorchClassifier for ART
classifier = PyTorchClassifier(model=net,
                               loss=criterion,
                               optimizer=optimizer,
                               input_shape=(3, 32, 32),
                               nb_classes=10)

In [6]:
from art.attacks.evasion import (FastGradientMethod,
                                 ProjectedGradientDescentPyTorch,
                                 AutoAttack,
                                 AutoProjectedGradientDescent,
                                 AutoConjugateGradient,
                                 CarliniLInfMethod,
                                 CarliniL2Method,
                                 NewtonFool,
                                 DeepFool,
                                 ElasticNet,
                                 FrameSaliencyAttack,
                                 HopSkipJump,
                                 BasicIterativeMethod)

# Initializations

Adversarial Attack Class

In [7]:
class AdversarialAttacks:
  def __init__(self, classifier, epsilon, eps_iter, norm, iterations, second_attack_iters):
    self.classifier = classifier
    self.epsilon = epsilon
    self.eps_iter = eps_iter
    self.norm = norm
    self.iterations = iterations
    self.second_attack_iters = second_attack_iters

  def init_attacker(self, attack_type, **kwargs):

    if attack_type=='fast_gradient_method':
        return FastGradientMethod(self.classifier,
                                eps=self.epsilon,
                                eps_step=self.eps_iter,
                                minimal=True,
                                norm=self.norm,
                                **kwargs)
    elif attack_type=='projected_gradient_descent':
        return ProjectedGradientDescentPyTorch(self.classifier,
                                             eps=self.epsilon,
                                             eps_step=self.eps_iter,
                                             max_iter=self.iterations,
                                             norm=self.norm,
                                             **kwargs)
    elif attack_type=='auto_attack':
        return AutoAttack(estimator=self.classifier,
                        eps=self.epsilon,
                        eps_step=self.eps_iter,
                        norm=self.norm)
    elif attack_type=='auto_projected_gradient_descent':
        return AutoProjectedGradientDescent(estimator=self.classifier,
                                          eps=self.epsilon,
                                          eps_step=self.eps_iter,
                                          norm=self.norm,
                                          max_iter=self.iterations,
                                          **kwargs)
    elif attack_type=='auto_conjugate_gradient':
        return AutoConjugateGradient(estimator=self.classifier,
                                   eps=self.epsilon,
                                   eps_step=self.eps_iter,
                                   norm=self.norm,
                                   max_iter=self.iterations,
                                   **kwargs)
    elif attack_type=='carlini_wagner_linf':
        return CarliniLInfMethod(self.classifier,
                               max_iter=self.second_attack_iters,
                               **kwargs)
    elif attack_type=='carlini_wagner_l2':
        return CarliniL2Method(self.classifier,
                               max_iter=self.second_attack_iters,
                               **kwargs)
    elif attack_type=='newton_fool':
        return NewtonFool(self.classifier,
                        max_iter=self.iterations,
                        **kwargs)
    elif attack_type=='deep_fool':
        return DeepFool(self.classifier,
                      max_iter=self.iterations,
                      epsilon=self.eps_iter,
                      **kwargs)
    elif attack_type=='elastic_net':
        return ElasticNet(self.classifier,
                      max_iter=self.second_attack_iters)
    elif attack_type=='frame_saliency':
        attacker = BasicIterativeMethod(self.classifier,
                                                 eps=self.epsilon,
                                                 eps_step=self.eps_iter,
                                                 max_iter=self.iterations,
                                      )
        return FrameSaliencyAttack(self.classifier,
                                 attacker,
                                 method='iterative_saliency')
    elif attack_type=='hop_skip_jump':
        return HopSkipJump(self.classifier,
                         norm=self.norm,
                         max_iter=self.second_attack_iters)
    else:
        raise ValueError(f'Attack type "{attack_type}" not supported!')

Adversarial Attack with Early Stopping

In [26]:
def attack_with_early_stopping(classifier, x, y, max_iterations, attacker, verbose: bool = True):
    label_flipped = False
    count = 0
    start_time = time.time()

    x = x.unsqueeze(0).numpy()
    y = y.numpy()

    outputs = classifier.predict(x)
    _, clean_predicted = torch.max(torch.tensor(outputs).data, 1)
     
    if clean_predicted.item()!=int(y):
        print('Misclassified input. Not attacking.')
        end_time = time.time()
        return x, end_time - start_time

    for i in range(max_iterations):

        adv_inputs = attacker.generate(x, y=np.expand_dims(y, axis=0))

        outputs = classifier.predict(adv_inputs)
        _, predicted = torch.max(torch.tensor(outputs).data, 1)

        label_flipped = bool(predicted.item()!=int(y))

        if label_flipped:
            if verbose:
              print(f'\nIterations for successful attack: {i+1}\n')
            break

        x = adv_inputs.copy()
    
    end_time = time.time()
    return adv_inputs, end_time - start_time

Adversarial Distance calculation for adversarial attack methods

In [28]:
def distance_calculation(classifier, x, y, epsilon, eps_iter, norm, max_iterations, attack_type, get_image: bool = False, verbose: bool = True):

    distance_list, runtime_list = [], []

    attacks = AdversarialAttacks(classifier=classifier,
                          epsilon=epsilon,
                          eps_iter=eps_iter,
                          norm=norm,
                          iterations=1,
                          second_attack_iters=40)
    attacker = attacks.init_attacker(attack_type)

    correct_prediction = 0

    for i, x in enumerate(xtest):
        x_adversarial, runtime = attack_with_early_stopping(classifier=classifier,
                                                            x=x,
                                                            y=y[i],
                                                            max_iterations=max_iterations,
                                                            attacker=attacker)

        x_adversarial_tensor = torch.from_numpy(x_adversarial)

        # Adversarial accuracy calculation
        output_adversarial = classifier.predict(x_adversarial)
        _, predicted_adversarial = torch.max(torch.tensor(output_adversarial).data, 1)
        correct_prediction += (predicted_adversarial.item() == int(y[i]))

        distance = torch.norm((x - x_adversarial_tensor), p=float(norm))
        distance_list.append(distance.item())
        runtime_list.append(runtime)

        if verbose:
            print(f'Image {i}\t\tAdversarial_distance: {distance:.5f}\t\tRuntime: {runtime:5f} seconds')

    if get_image:
        get_example_image(x_adversarial, predicted_adversarial.item(), attack_type=attack_type)
        get_example_image(x.unsqueeze(0).numpy(), y[i], attack_type='original')
    
    adversarial_accuracy = (correct_prediction / len(xtest)) * 100
    print(f'\nAdversarial accuracy: {adversarial_accuracy}%\n')

    return distance_list, runtime_list, adversarial_accuracy

Generate and Save Example Images

In [22]:
def get_example_image(x, y, attack_type: str):
    label_map = {0: 'airplane',
               1: 'automobile',
               2: 'bird',
               3: 'cat',
               4: 'deer',
               5: 'dog',
               6: 'frog',
               7: 'horse',
               8: 'ship',
               9: 'truck'}

    img = x.squeeze(0).transpose(1, 2, 0)
    img = Image.fromarray((img * 255).astype('uint8'), 'RGB')
    img = img.resize((224, 224))

    img = ImageOps.expand(img, border=10, fill=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    title = f'Label: {label_map[int(y)]}'
    draw.text((0, 0), title, (0, 0, 0))

    img.save(f'./data/example_image_{attack_type}.png')

Model configuration and Hyperparameter specifications

In [23]:
norm = 2
max_iterations = 500
eps_iter_dict = {
    'inf': 0.0003,
    '1': 0.2,
    '2': 0.005}
eps_iter = eps_iter_dict[str(norm)]
epsilon = max_iterations * eps_iter

Load CIFAR-10 Test Dataset

In [24]:
splitsize = 5
xtest, ytest = load_dataset(dataset_split=splitsize)

Files already downloaded and verified


# Evaluation

In [29]:
attack_types = [
                'fast_gradient_method',
                'projected_gradient_descent',
#                 'auto_projected_gradient_descent',
#                 'auto_conjugate_gradient',
                'newton_fool',
                'deep_fool',
#                 'elastic_net',
#                 'frame_saliency',
#                 'auto_attack',
#                 'carlini_wagner_linf',
#                 'carlini_wagner_l2',
#                 'hop_skip_jump'
                ]

results_dict = {}

for attack_type in attack_types:
    results_dict[attack_type] = {}
    print(f'\t\t-------------------------- Processing Attack: {attack_type} --------------------------\n')
    results_dict[attack_type]["adversarial_distance"], results_dict[attack_type]["runtime"], results_dict[attack_type]["adversarial_accuracy"] = distance_calculation(classifier=classifier,
                                                        x=xtest,
                                                        y=ytest,
                                                        epsilon=epsilon,
                                                        eps_iter=eps_iter,
                                                        norm=norm,
                                                        max_iterations=max_iterations,
                                                        attack_type=attack_type)
    
    print(f'\nMean adversarial distance for {attack_type}: {np.mean(results_dict[attack_type]["adversarial_distance"]):.5f} with total runtime: {sum(results_dict[attack_type]["runtime"]): .5f} seconds\n')

		-------------------------- Processing Attack: fast_gradient_method --------------------------


Iterations for successful attack: 1

Image 0		Adversarial_distance: 0.32500		Runtime: 0.361722 seconds

Iterations for successful attack: 1

Image 1		Adversarial_distance: 2.02500		Runtime: 2.391035 seconds

Iterations for successful attack: 1

Image 2		Adversarial_distance: 1.08500		Runtime: 1.249772 seconds

Iterations for successful attack: 1

Image 3		Adversarial_distance: 0.33000		Runtime: 0.421917 seconds

Iterations for successful attack: 1

Image 4		Adversarial_distance: 0.17000		Runtime: 0.207507 seconds

Adversarial accuracy: 0.0%


Mean adversarial distance for fast_gradient_method: 0.78700 with total runtime:  4.63195 seconds

		-------------------------- Processing Attack: projected_gradient_descent --------------------------



PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...


Iterations for successful attack: 2

Image 2		Adversarial_distance: 0.59043		Runtime: 0.374093 seconds


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]


Iterations for successful attack: 1

Image 3		Adversarial_distance: 0.81802		Runtime: 0.173673 seconds


DeepFool:   0%|          | 0/1 [00:00<?, ?it/s]


Iterations for successful attack: 1

Image 4		Adversarial_distance: 0.37193		Runtime: 0.191754 seconds

Adversarial accuracy: 0.0%


Mean adversarial distance for deep_fool: 0.65132 with total runtime:  1.46360 seconds



Save Results to JSON File

In [30]:
import json

json_file_path = f'./data/attack_comparison_{modeltype}_L{norm}.json'

with open(json_file_path, 'w') as f:
    json.dump(results_dict, f, indent=4)
print(f'Evaluation results are saved under "{json_file_path}".')

Evaluation results are saved under "./data/attack_comparison_robust_L2.json".


In [ ]:
# plt.figure(figsize=(15, 8))
# for attack_type in attack_types:
#   plt.scatter(list(range(len(xtest))), results_dict[attack_type]['adversarial_distance'], label=attack_type)
# plt.xlabel('Image ID ----->')
# plt.ylabel('Distance ----->')
# plt.title(f'L_{norm} Distance')
# plt.legend()
# plt.tight_layout()
# plt.show()

In [ ]:
# plt.figure(figsize=(15, 8))
# for attack_type in attack_types:
#   plt.plot(list(range(len(xtest))), results_dict[attack_type]['runtime'], label=attack_type)
# plt.xlabel('Image ID ----->')
# plt.ylabel('Runtime [seconds] ----->')
# plt.title('Step Runtime')
# plt.legend()
# plt.tight_layout()
# plt.show()

In [ ]:
# plt.figure(figsize=(15, 8))
# for attack_type in attack_types:
#   plt.bar(attack_type, results_dict[attack_type]['adversarial_accuracy'], label=attack_type)
# plt.xlabel('Attacks')
# plt.ylabel('Adversarial accuracy [%] ----->')
# plt.xticks(rotation=45)
# plt.legend()
# plt.title('Adversarial Accuracy')
# plt.tight_layout()
# plt.show()

In [ ]:
# plt.figure(figsize=(15, 8))
# for attack_type in attack_types:
#   plt.bar(attack_type, sum(results_dict[attack_type]['runtime']), label=attack_type)
# plt.xlabel('Attacks')
# plt.xticks(rotation=45)
# plt.ylabel('Runtime [seconds]----->')
# plt.title('Total Runtime')
# plt.legend()
# plt.tight_layout()
# plt.show()